In [1]:
import numpy as np
import respy as rp
import pandas as pd

/home/maxbl/Uni/Robustness/structRobustness/sandbox/submodules/respy/respy/_numba.py:2: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.cgutils', please update to use 'numba.core.cgutils' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba import cgutils


In [2]:
process_name = "health_shock"
process_states = ["good", "bad"]

In [3]:
df_exog = pd.DataFrame(
    columns=process_states,
    index=["sick_and_old", "sick_and_young", "healthy_and_old", "healthy_and_young"],
    data=[[0.8, 0.2],
          [0.6, 0.4],
          [0.3, 0.7],
          [0.4, 0.6]],
    dtype=float
)
df_exog

,good,bad
sick_and_old,0.8,0.2
sick_and_young,0.6,0.4
healthy_and_old,0.3,0.7
healthy_and_young,0.4,0.6


In [4]:
params_exog, covs_exog = rp.parse_transition_matrix_for_exogenous_processes(df_exog, process_name)

In [5]:
params_exog.dtypes

value    float64
dtype: object

In [6]:
covs_exog

{'sick_and_old': '?',
 'sick_and_young': '?',
 'healthy_and_old': '?',
 'healthy_and_young': '?'}

In [7]:
covs_exog['sick'] = "health_shock == 'sick'"
covs_exog['healthy'] = "health_shock == 'healthy'"
covs_exog['sick_and_old'] = "sick & old"
covs_exog['sick_and_young'] = "sick & young"
covs_exog['healthy_and_old'] = "healthy & old"
covs_exog['healthy_and_young'] = "healthy & young"
covs_exog

{'sick_and_old': 'sick & old',
 'sick_and_young': 'sick & young',
 'healthy_and_old': 'healthy & old',
 'healthy_and_young': 'healthy & young',
 'sick': "health_shock == 'sick'",
 'healthy': "health_shock == 'healthy'"}

In [8]:
covs_exog = {**covs_exog, "age": "16 + period",
                        "old": "age > 50",
                        "young": "age <= 50"}
covs_exog

{'sick_and_old': 'sick & old',
 'sick_and_young': 'sick & young',
 'healthy_and_old': 'healthy & old',
 'healthy_and_young': 'healthy & young',
 'sick': "health_shock == 'sick'",
 'healthy': "health_shock == 'healthy'",
 'age': '16 + period',
 'old': 'age > 50',
 'young': 'age <= 50'}

In [9]:
params, options = rp.get_example_model("kw_94_one", with_data=False)

In [10]:
params

value  \
category                name                                   
delta                   delta                         0.9500   
wage_a                  constant                      9.2100   
                        exp_edu                       0.0380   
                        exp_a                         0.0330   
                        exp_a_square                 -0.0005   
                        exp_b                         0.0000   
                        exp_b_square                  0.0000   
wage_b                  constant                      8.4800   
                        exp_edu                       0.0700   
                        exp_b                         0.0670   
                        exp_b_square                 -0.0010   
                        exp_a                         0.0220   
                        exp_a_square                 -0.0005   
nonpec_edu              constant                      0.0000   
                        at_least_twelve_exp_edu       0.0000   
                        not_edu_last_period       -4000.0000   
nonpec_home             constant                  17750.0000   
shocks_sdcorr           sd_a                          0.2000   
                        sd_b                          0.2500   
                        sd_edu                     1500.0000   
                        sd_home                    1500.0000   
                        corr_b_a                      0.0000   
                        corr_edu_a                    0.0000   
                        corr_edu_b                    0.0000   
                        corr_home_a                   0.0000   
                        corr_home_b                   0.0000   
                        corr_home_edu                 0.0000   
lagged_choice_1_edu     probability                   1.0000   
initial_exp_edu_10      probability                   1.0000   
maximum_exp             edu                          20.0000   
inadmissibility_penalty inadmissibility_penalty -400000.0000   

                                                                                           comment  
category                name                                                                        
delta                   delta                                                      discount factor  
wage_a                  constant                                               log of rental price  
                        exp_edu                          return to an additional year of schooling  
                        exp_a                                     return to same sector experience  
                        exp_a_square                   return to same sector, quadratic experience  
                        exp_b                                    return to other sector experience  
                        exp_b_square                  return to other sector, quadratic experience  
wage_b                  constant                                               log of rental price  
                        exp_edu                          return to an additional year of schooling  
                        exp_b                                     return to same sector experience  
                        exp_b_square                   return to same sector, quadratic experience  
                        exp_a                                    return to other sector experience  
                        exp_a_square                  return to other sector, quadratic experience  
nonpec_edu              constant                            constant reward for choosing education  
                        at_least_twelve_exp_edu        reward for going to college (tuition, etc.)  
                        not_edu_last_period                        reward for going back to school  
nonpec_home             constant                         constant reward of non-market alternative  
shocks_sdcorr          

In [11]:
options["covariates"] = {**options["covariates"], **covs_exog}
options

{'estimation_draws': 200,
 'estimation_seed': 500,
 'estimation_tau': 500,
 'interpolation_points': -1,
 'n_periods': 40,
 'simulation_agents': 1000,
 'simulation_seed': 132,
 'solution_draws': 500,
 'solution_seed': 3,
 'monte_carlo_sequence': 'random',
 'core_state_space_filters': ["period > 0 and exp_{i} == period and lagged_choice_1 != '{i}'",
  "period > 0 and exp_a + exp_b + exp_edu == period and lagged_choice_1 == '{j}'",
  "period > 0 and lagged_choice_1 == 'edu' and exp_edu == 0",
  "lagged_choice_1 == '{k}' and exp_{k} == 0",
  "period == 0 and lagged_choice_1 == '{k}'"],
 'covariates': {'constant': '1',
  'exp_a_square': 'exp_a ** 2',
  'exp_b_square': 'exp_b ** 2',
  'at_least_twelve_exp_edu': 'exp_edu >= 12',
  'not_edu_last_period': "lagged_choice_1 != 'edu'",
  'sick_and_old': 'sick & old',
  'sick_and_young': 'sick & young',
  'healthy_and_old': 'healthy & old',
  'healthy_and_young': 'healthy & young',
  'sick': "health_shock == 'sick'",
  'healthy': "health_shock == '

In [12]:
params.loc[("wage_a", "sick"), "value"] = -0.2
params.loc[("wage_b", "sick"), "value"] = -0.4
params.loc[("nonpec_edu", "sick"), "value"] = -0.1
params.loc[("nonpec_home", "sick"), "value"] = 5
params.loc[("observable_health_shock_healthy", "probability"), "value"] = 1
params.loc[("observable_health_shock_sick", "probability"), "value"] = 0
params_final = pd.concat([params, params_exog])

In [13]:
params_final

value  \
category                               name                                     
delta                                  delta                         0.950000   
wage_a                                 constant                      9.210000   
                                       exp_edu                       0.038000   
                                       exp_a                         0.033000   
                                       exp_a_square                 -0.000500   
                                       exp_b                         0.000000   
                                       exp_b_square                  0.000000   
wage_b                                 constant                      8.480000   
                                       exp_edu                       0.070000   
                                       exp_b                         0.067000   
                                       exp_b_square                 -0.001000   
                                       exp_a                         0.022000   
                                       exp_a_square                 -0.000500   
nonpec_edu                             constant                      0.000000   
                                       at_least_twelve_exp_edu       0.000000   
                                       not_edu_last_period       -4000.000000   
nonpec_home                            constant                  17750.000000   
shocks_sdcorr                          sd_a                          0.200000   
                                       sd_b                          0.250000   
                                       sd_edu                     1500.000000   
                                       sd_home                    1500.000000   
                                       corr_b_a                      0.000000   
                                       corr_edu_a                    0.000000   
                                       corr_edu_b                    0.000000   
                                       corr_home_a                   0.000000   
                                       corr_home_b                   0.000000   
                                       corr_home_edu                 0.000000   
lagged_choice_1_edu                    probability                   1.000000   
initial_exp_edu_10                     probability                   1.000000   
maximum_exp                            edu                          20.000000   
inadmissibility_penalty                inadmissibility_penalty -400000.000000   
wage_a                                 sick                         -0.200000   
wage_b                                 sick                         -0.400000   
nonpec_edu                             sick                         -0.100000   
nonpec_home                            sick                          5.000000   
observable_health_shock_healthy        probability                   1.000000   
observable_health_shock_sick           probability                   0.000000   
exogenous_process_health_shock_healthy sick_and_old                 -0.223144   
                                       sick_and_young               -0.510826   
                                       healthy_and_old              -1.203973   
                                       healthy_and_young            -0.916291   
exogenous_process_health_shock_sick    sick_and_old                 -1.609438   
                                       sick_and_young               -0.916291   
                                       healthy_and_old              -0.356675   
                                       healthy_and_young            -0.510826   

                                                                                                          comment  
category                               name                                                                        
delta                           

In [14]:
options

{'estimation_draws': 200,
 'estimation_seed': 500,
 'estimation_tau': 500,
 'interpolation_points': -1,
 'n_periods': 40,
 'simulation_agents': 1000,
 'simulation_seed': 132,
 'solution_draws': 500,
 'solution_seed': 3,
 'monte_carlo_sequence': 'random',
 'core_state_space_filters': ["period > 0 and exp_{i} == period and lagged_choice_1 != '{i}'",
  "period > 0 and exp_a + exp_b + exp_edu == period and lagged_choice_1 == '{j}'",
  "period > 0 and lagged_choice_1 == 'edu' and exp_edu == 0",
  "lagged_choice_1 == '{k}' and exp_{k} == 0",
  "period == 0 and lagged_choice_1 == '{k}'"],
 'covariates': {'constant': '1',
  'exp_a_square': 'exp_a ** 2',
  'exp_b_square': 'exp_b ** 2',
  'at_least_twelve_exp_edu': 'exp_edu >= 12',
  'not_edu_last_period': "lagged_choice_1 != 'edu'",
  'sick_and_old': 'sick & old',
  'sick_and_young': 'sick & young',
  'healthy_and_old': 'healthy & old',
  'healthy_and_young': 'healthy & young',
  'sick': "health_shock == 'sick'",
  'healthy': "health_shock == '

In [15]:
options["covariates"]

{'constant': '1',
 'exp_a_square': 'exp_a ** 2',
 'exp_b_square': 'exp_b ** 2',
 'at_least_twelve_exp_edu': 'exp_edu >= 12',
 'not_edu_last_period': "lagged_choice_1 != 'edu'",
 'sick_and_old': 'sick & old',
 'sick_and_young': 'sick & young',
 'healthy_and_old': 'healthy & old',
 'healthy_and_young': 'healthy & young',
 'sick': "health_shock == 'sick'",
 'healthy': "health_shock == 'healthy'",
 'age': '16 + period',
 'old': 'age > 50',
 'young': 'age <= 50'}

In [20]:
simulate = rp.get_simulate_func(params_final, options)
df = simulate(params)

In [24]:
df

Experience_A  Experience_B  Experience_Edu Lagged_Choice_1  \
Identifier Period                                                               
0          0                  0             0              10             edu   
           1                  1             0              10               a   
           2                  2             0              10               a   
           3                  3             0              10               a   
           4                  4             0              10               a   
...                         ...           ...             ...             ...   
999        35                 1            27              17               b   
           36                 1            28              17               b   
           37                 1            29              17               b   
           38                 1            30              17               b   
           39                 1            31              17               b   

                  Health_Shock  Shock_Reward_A  Meas_Error_Wage_A  \
Identifier Period                                                   
0          0           healthy        1.154231                  1   
           1           healthy        0.979806                  1   
           2           healthy        1.230656                  1   
           3           healthy        1.421402                  1   
           4           healthy        0.736126                  1   
...                        ...             ...                ...   
999        35          healthy        1.350223                  1   
           36          healthy        0.980942                  1   
           37          healthy        1.043932                  1   
           38          healthy        1.457091                  1   
           39          healthy        1.057029                  1   

                   Shock_Reward_B  Meas_Error_Wage_B  Shock_Reward_Edu  ...  \
Identifier Period                                                       ...   
0          0             1.293855                  1      -2875.095456  ...   
           1             0.909449                  1      -1620.841705  ...   
           2             0.947062                  1       1051.039033  ...   
           3             1.068773                  1      -1266.157510  ...   
           4             0.856910                  1       3043.133417  ...   
...                           ...                ...               ...  ...   
999        35            1.192421                  1       -352.822830  ...   
           36            0.877348                  1       -776.089588  ...   
           37            1.051058                  1      -1588.446803  ...   
           38            1.498542                  1      -1199.909239  ...   
           39            0.770794                  1      -1357.467037  ...   

                   Nonpecuniary_Reward_Edu  Wage_Edu  Flow_Utility_Edu  \
Identifier Period                                                        
0          0                             0       NaN      -2875.095456   
           1                         -4000       NaN      -5620.841705   
           2                         -4000       NaN      -2948.960967   
           3                         -4000       NaN      -5266.157510   
           4                         -4000       NaN       -956.866583   
...                                    ...       ...               ...   
999        35                        -4000       NaN      -4352.822830   
           36                        -4000       NaN      -4776.089588   
           37                        -4000       NaN      -5588.446803   
           38                        -4000       NaN      -5199.909239   
           39                        -4000       NaN      -5357.467037   

                  Value_Function_Edu  Continuation_Value_Edu  \
Identifier P